In [13]:
import pandas as pd
import os
import json

pid_labels = json.load(open(os.path.join("..", "data", "pid_labels_persons.json")))
reference_labels = json.load(open(os.path.join("..", "data", "pid_labels.json")))
pid_labels = {**pid_labels, **reference_labels}
pid_labels["P887"] = "based on heuristic"
pid_labels = {key: f"{value} ({key})" for key, value in pid_labels.items()}


filter_list = {key for key, label in pid_labels.items() if "ID" in label or "ISNI" in label or "category" in label.lower()}

df_pid_counter = pd.read_pickle(os.path.join("..", "data", "pid_counter.pkl"))
df_claim_counter = pd.read_pickle(os.path.join("..", "data", "claim_counter.pkl"))
df_claim_with_ref_counter = pd.read_pickle(os.path.join("..", "data", "claim_with_ref_counter.pkl"))
df_num_refs = pd.read_pickle(os.path.join("..", "data", "num_refs.pkl"))
df_num_refs_detailed = pd.read_pickle(os.path.join("..", "data", "num_refs_detailed.pkl"))
df_num_refs_with_url_detailed = pd.read_pickle(os.path.join("..", "data", "num_refs_with_url_detailed.pkl"))
df_num_refs_per_pid = pd.read_pickle(os.path.join("..", "data", "num_refs_per_pid.pkl"))

def remove_wrong_decades(df):
    try:
        return df[df['decade'].str.extract(r'(\d{4})').astype(int)[0] <= 2020]
    except AttributeError:
        return df[df[('decade', None)].str.extract(r'(\d{4})').astype(int)[0] <= 2020]
df_pid_counter = remove_wrong_decades(df_pid_counter)
df_claim_counter = remove_wrong_decades(df_claim_counter)
df_claim_with_ref_counter = remove_wrong_decades(df_claim_with_ref_counter)
df_num_refs = remove_wrong_decades(df_num_refs)
df_num_refs_detailed = remove_wrong_decades(df_num_refs_detailed)
df_num_refs_with_url_detailed = remove_wrong_decades(df_num_refs_with_url_detailed)
df_num_refs_per_pid = remove_wrong_decades(df_num_refs_per_pid)


In [14]:

num_elements_to_consider = 50
p_cols = df_pid_counter.columns[df_pid_counter.columns.str.startswith('P')]
most_popular_pids = df_pid_counter[p_cols].sum(axis=0).nlargest(num_elements_to_consider).index.tolist()
most_popular_pids = [pid for pid in most_popular_pids if pid in pid_labels]
most_popular_pids = [pid for pid in most_popular_pids if pid not in filter_list]

# Step 1: Select columns where COUNT > 1 (second level of column MultiIndex)
filtered = df_num_refs_detailed.loc[:, df_num_refs_detailed.columns.get_level_values(1) > 1]
# Step 2: Sum across rows (axis=0) for each PID — group by first level of column MultiIndex
result = filtered.groupby(axis=1, level=0).sum()

df_claim_counter_summed = df_claim_counter.loc[:, df_claim_counter.columns.str.startswith('P')]
df_claim_counter_summed = df_claim_counter_summed.sum()
p_cols = result.columns[result.columns.str.startswith('P')]
summed  = result[p_cols].sum(axis=0)
for pid in summed.index:
    if not pid.startswith("P"):
        continue
    if pid in df_claim_counter_summed.index:
        summed[pid] = summed[pid] / df_claim_counter_summed.loc[pid]
most_popular_pids_with_more_than_one_ref = summed.nlargest(num_elements_to_consider).index.tolist()
most_popular_pids_with_more_than_one_ref = [pid for pid in most_popular_pids_with_more_than_one_ref if pid in pid_labels]
most_popular_pids_with_more_than_one_ref = [pid for pid in most_popular_pids_with_more_than_one_ref if pid not in filter_list]
most_popular_ref_pids = df_num_refs_per_pid.groupby(level=1, axis=1).sum()
most_popular_ref_pids = most_popular_ref_pids.sum(axis=0).nlargest(50).index.tolist()
most_popular_ref_pids = ["P854", "P4656", "P248", "P3452", "P813", "P887"]
print("Most popular PIDs: ", [pid_labels.get(pid) for pid in most_popular_pids])
print("Most popular PIDs with more than one ref: ", [pid_labels.get(pid) for pid in most_popular_pids_with_more_than_one_ref])
print("Most popular reference PIDs: ", [pid_labels.get(pid, pid) for pid in most_popular_ref_pids])


/var/folders/l7/zj7563zj4nv2znvkzcllpvvr0000gn/T/ipykernel_56126/3585352213.py:10: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  result = filtered.groupby(axis=1, level=0).sum()
/var/folders/l7/zj7563zj4nv2znvkzcllpvvr0000gn/T/ipykernel_56126/3585352213.py:24: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  most_popular_ref_pids = df_num_refs_per_pid.groupby(level=1, axis=1).sum()


Most popular PIDs:  ['sex or gender (P21)', 'instance of (P31)', 'date of birth (P569)', 'country of citizenship (P27)', 'occupation (P106)', 'given name (P735)', 'place of birth (P19)', 'family name (P734)', 'languages spoken, written or signed (P1412)', 'sport (P641)', 'educated at (P69)', 'image (P18)', 'date of death (P570)', 'member of sports team (P54)', 'name in native language (P1559)', 'position played on team / speciality (P413)', 'position held (P39)', 'employer (P108)', 'award received (P166)', 'member of political party (P102)', 'place of death (P20)', 'height (P2048)', 'participant in (P1344)', 'native language (P103)', 'work period (start) (P2031)', 'country for sport (P1532)', 'X (Twitter) username (P2002)', 'official website (P856)', 'mass (P2067)']
Most popular PIDs with more than one ref:  ['location of discovery (P189)', 'manufacturer (P176)', 'autonomous system number (P3797)', 'e-Rad researcher number (P9776)', 'post office box (P2918)', 'Wikidata item of this pro

In [15]:


from IPython.core.display import Markdown


def calculate_all_stats(df_pid_counter, df_claim_counter , df_claim_with_ref_counter, df_num_refs, df_num_refs_per_pid, group_by: list):
    # Accumulate over genders
    df_pid_counter = df_pid_counter.groupby(group_by).sum()
    df_claim_counter = df_claim_counter.groupby(group_by).sum()
    df_claim_with_ref_counter = df_claim_with_ref_counter.groupby(group_by).sum()
    df_num_refs = df_num_refs.groupby(group_by).sum()
    df_num_refs_per_pid = df_num_refs_per_pid.groupby([(x, None) for x in group_by]).sum()

    p_cols = df_pid_counter.columns[df_pid_counter.columns.str.startswith('P')]
    df_pid_counter[p_cols] = df_pid_counter[p_cols].div(df_pid_counter['person_counter'], axis=0)

    def create_pid_stats(filter_pids):
        # Filter all PID columns out that are not in the most_popular_pids
        df_pid_counter_most_pop = df_pid_counter.loc[:, df_pid_counter.columns.isin(filter_pids)]
        df_pid_counter_most_pop.fillna(0)
        df_pid_counter_most_pop = df_pid_counter_most_pop[df_pid_counter_most_pop.iloc[0].sort_values(ascending=False).index]
        df_pid_counter_most_pop = df_pid_counter_most_pop.rename(columns=pid_labels)
        return df_pid_counter_most_pop

    def create_ref_stats(filter_pids, normalize_by):
        normalized_num_refs_per_pid = df_num_refs_per_pid.copy()
        for pid in df_num_refs_per_pid.columns:
            if not pid[0].startswith("P"):
                continue
            if pid[0] in normalize_by.columns:
                normalized_num_refs_per_pid[pid] = normalized_num_refs_per_pid[pid].div(normalize_by[pid[0]], axis=0)

        normalized_num_refs_per_pid = normalized_num_refs_per_pid.fillna(0)
        normalized_num_refs_per_pid = normalized_num_refs_per_pid.loc[:, normalized_num_refs_per_pid.columns.get_level_values(0).isin(filter_pids)]
        normalized_num_refs_per_pid = normalized_num_refs_per_pid.groupby(level=1, axis=1).mean()
        normalized_num_refs_per_pid = normalized_num_refs_per_pid.loc[:, normalized_num_refs_per_pid.columns.isin(most_popular_ref_pids)]
        normalized_num_refs_per_pid = normalized_num_refs_per_pid[normalized_num_refs_per_pid.iloc[0].sort_values(ascending=False).index]
        normalized_num_refs_per_pid = normalized_num_refs_per_pid.rename(columns=pid_labels)
        return normalized_num_refs_per_pid

    display(Markdown(f"# Group by {group_by}"))
    display(Markdown("## Calculate percentage of persons with specific relation having references (there might be multiple claims per person)"))
    copy_df_num_refs = df_claim_with_ref_counter.copy()
    p_cols = copy_df_num_refs.columns[copy_df_num_refs.columns.str.startswith('P')]
    copy_df_num_refs[p_cols] = copy_df_num_refs[p_cols].div(df_num_refs['person_counter'], axis=0)
    copy_df_num_refs = copy_df_num_refs.rename(columns=pid_labels)
    display(copy_df_num_refs)

    display(Markdown("## Calculate percentage of claims with specific relation having references"))
    copy_df_num_refs = df_claim_with_ref_counter.copy()
    for pid in copy_df_num_refs.columns:
            if not pid.startswith("P"):
                continue
            if pid in df_claim_counter.columns:
                copy_df_num_refs[pid] = copy_df_num_refs[pid].div(df_claim_counter[pid], axis=0)
    copy_df_num_refs = copy_df_num_refs.rename(columns=pid_labels)
    display(copy_df_num_refs)

    display(Markdown("## Calculate percentage of claims having references"))
    copy_df_claim_with_ref_counter = df_claim_with_ref_counter.copy()
    p_columns = [col for col in copy_df_claim_with_ref_counter.columns if col.startswith('P')]
    # Sum those columns row-wise
    copy_df_claim_with_ref_counter['P_sum'] = copy_df_claim_with_ref_counter[p_columns].sum(axis=1)
    # Drop the original 'P' columns
    df_claim_with_ref_summed = copy_df_claim_with_ref_counter.drop(columns=p_columns)

    copy_df_claim_counter = df_claim_counter.copy()
    p_columns = [col for col in copy_df_claim_counter.columns if col.startswith('P')]
    # Sum those columns row-wise
    copy_df_claim_counter['P_sum'] = copy_df_claim_counter[p_columns].sum(axis=1)
    # Drop the original 'P' columns
    df_claim_summed = copy_df_claim_counter.drop(columns=p_columns)
    df_normalized_claim_with_ref = df_claim_with_ref_summed.copy()
    for pid in df_claim_with_ref_summed.columns:
        if not pid.startswith("P"):
            continue
        if pid in df_claim_summed.columns:
            df_normalized_claim_with_ref[pid] = df_claim_with_ref_summed[pid].div(df_claim_summed[pid], axis=0)

    display(df_normalized_claim_with_ref)


    display(Markdown("## Calculate percentage of claims having a specific reference type"))
    copy_df_claim_with_ref_counter = df_num_refs_per_pid.copy()
    df_summed = copy_df_claim_with_ref_counter.groupby(level=1, axis=1).sum()
    for pid in copy_df_num_refs.columns:
            if not pid.startswith("P"):
                continue
            if pid in df_claim_counter.columns:
                copy_df_num_refs[pid] = copy_df_num_refs[pid].div(df_claim_counter[pid], axis=0)
    df_summed = df_summed.loc[:, df_summed.columns.isin(most_popular_ref_pids)]
    # Normalize by df_claim_summed P_sum value for each corresponding row
    for pid in df_summed.columns:
        if not pid.startswith("P"):
            continue
        df_summed[pid] = df_summed[pid].div(copy_df_claim_counter["P_sum"], axis=0)
    df_summed = df_summed.rename(columns=pid_labels)
    display(df_summed)

    return

    display(Markdown(f"## Calculate the average occurrence of each relation"))
    display(Markdown("### For each decade and the top-50 most popular relations (if multiple claims occur, it is still counted as one - replacing with df_claim_counter would consider this)"))
    df_top_pop = create_pid_stats(most_popular_pids)
    display(df_top_pop)
    if len(group_by) > 1:
        display(Markdown("### Average over all decades for the top-50 most popular relations"))
        df_top_pop_averaged = df_top_pop.groupby(["income_class"]).mean()
        display(df_top_pop_averaged)
    display(Markdown("### For each decade and the top-50 relations that are equipped with the most references (not just counting the claims with references but the ratio of claims with references to all claims)"))
    df_top_ref_pop = create_pid_stats(most_popular_pids_with_more_than_one_ref)
    display(df_top_ref_pop)
    if len(group_by) > 1:
        display(Markdown("### Average over all decades for the top-50 relations that are equipped with the most references"))
        df_top_ref_pop_averaged = df_top_ref_pop.groupby(["income_class"]).mean()
        display(df_top_ref_pop_averaged)
    display(Markdown("## Calculate average use of references"))
    display(Markdown("### For each decade and the top-50 most popular relations - normalized by claims"))
    df_ref_stats_top_pop_claim = create_ref_stats(most_popular_pids, df_claim_counter)
    display(df_ref_stats_top_pop_claim)
    if len(group_by) > 1:
        display(Markdown("### Average over all decades for the top-50 most popular relations - normalized by claims"))
        df_ref_stats_top_pop_claim_averaged = df_ref_stats_top_pop_claim.groupby(level=0).mean()
        display(df_ref_stats_top_pop_claim_averaged)
    display(Markdown("### For each decade and the top-50 relations that are equipped with the most references - normalized by claims"))
    df_ref_stats_top_ref_pop_claim = create_ref_stats(most_popular_pids_with_more_than_one_ref, df_claim_counter)
    display(df_ref_stats_top_ref_pop_claim)
    if len(group_by) > 1:
        display(Markdown("### Average over all decades for the top-50 relations that are equipped with the most references - normalized by claims"))
        df_ref_stats_top_ref_pop_claim_averaged = df_ref_stats_top_ref_pop_claim.groupby(level=0).mean()
        display(df_ref_stats_top_ref_pop_claim_averaged)
    display(Markdown("### For each decade and the top-50 relations - normalized by claims with references"))
    df_ref_stats_top_pop_claim_ref = create_ref_stats(most_popular_pids, df_claim_with_ref_counter)
    display(df_ref_stats_top_pop_claim_ref)
    if len(group_by) > 1:
        display(Markdown("### Average over all decades for the top-50 relations - normalized by claims with references"))
        df_ref_stats_top_pop_claim_ref_averaged = df_ref_stats_top_pop_claim_ref.groupby(level=0).mean()
        display(df_ref_stats_top_pop_claim_ref_averaged)
    display(Markdown("### For each decade and the top-50 relations that are equipped with the most references - normalized by claims with references"))
    df_ref_stats_top_ref_pop_claim_ref = create_ref_stats(most_popular_pids_with_more_than_one_ref, df_claim_with_ref_counter)
    display(df_ref_stats_top_ref_pop_claim_ref)
    if len(group_by) > 1:
        display(Markdown("### Average over all decades for the top-50 relations that are equipped with the most references - normalized by claims with references"))
        df_ref_stats_top_ref_pop_claim_ref_averaged = df_ref_stats_top_ref_pop_claim_ref.groupby(level=0).mean()
        display(df_ref_stats_top_ref_pop_claim_ref_averaged)





In [16]:
calculate_all_stats(df_pid_counter, df_claim_counter, df_claim_with_ref_counter, df_num_refs, df_num_refs_per_pid, group_by=['income_class'])

# Group by ['income_class']

## Calculate percentage of persons with specific relation having references (there might be multiple claims per person)

,person_counter,decade,gender,VIAF cluster ID (P214),member of political party (P102),educated at (P69),instance of (P31),place of birth (P19),place of death (P20),position held (P39),...,Encyclopedia of China (Third Edition) ID (P10565),member of cabinet (P5054),Ministry of Unification politician ID (P11150),has goal (P3712),parent club (P831),time played (P9140),The Counted person ID (P11638),measured physical quantity (P111),Netherlands Olympic Committee athlete ID (P10957),IMDb keyword (P11924)
income_class,,,,,,,,,,,,,,,,,,,,,
high_income,1847883,1930s1930s1920s1980s1990s1970s1950s1980s1960s1...,malefemalemalemalemalemalemalefemalemalemalefe...,0.214847,0.044028,0.269457,0.469116,0.528624,0.072912,0.221348,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
low_income,21533,1980s1960s1970s2000s1940s1950s1990s1970s1980s1...,malemalemalefemalemalemalemalefemalefemalefema...,0.085311,0.090652,0.185715,0.432545,0.488924,0.067339,0.219709,...,0.000000,0.000000,0.001347,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046
lower-middle_income,128003,1940s1960s1930s2000s1970s1980s1980s1940s2000s1...,malemalemalemalemalemalefemalefemalefemalemale...,0.092771,0.100037,0.234502,0.434857,0.515183,0.071795,0.153379,...,0.000008,0.000008,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
upper-middle_income,408665,2000s1940s1960s1950s1970s1970s1940s1980s1990s1...,femalemalemalemalefemalemalefemalemalemalemale...,0.066404,0.223366,0.160151,0.381868,0.545158,0.060113,0.079219,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Calculate percentage of claims with specific relation having references

,person_counter,decade,gender,VIAF cluster ID (P214),member of political party (P102),educated at (P69),instance of (P31),place of birth (P19),place of death (P20),position held (P39),...,Encyclopedia of China (Third Edition) ID (P10565),member of cabinet (P5054),Ministry of Unification politician ID (P11150),has goal (P3712),parent club (P831),time played (P9140),The Counted person ID (P11638),measured physical quantity (P111),Netherlands Olympic Committee athlete ID (P10957),IMDb keyword (P11924)
income_class,,,,,,,,,,,,,,,,,,,,,
high_income,1847883,1930s1930s1920s1980s1990s1970s1950s1980s1960s1...,malefemalemalemalemalemalemalefemalemalemalefe...,0.555894,0.513562,0.571728,0.468990,0.774240,0.746814,0.749961,...,0.000000,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0
low_income,21533,1980s1960s1970s2000s1940s1950s1990s1970s1980s1...,malemalemalefemalemalemalemalefemalefemalefema...,0.522172,0.699140,0.511446,0.432345,0.731213,0.682353,0.660201,...,0.000000,NaN,0.508772,1.0,0.5,1.0,1.0,1.0,1.0,1.0
lower-middle_income,128003,1940s1960s1930s2000s1970s1980s1980s1940s2000s1...,malemalemalemalemalemalefemalefemalefemalemale...,0.521085,0.640154,0.546151,0.434752,0.759656,0.666763,0.483179,...,0.003831,0.5,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
upper-middle_income,408665,2000s1940s1960s1950s1970s1970s1940s1980s1990s1...,femalemalemalemalefemalemalefemalemalemalemale...,0.490156,0.886655,0.568238,0.381806,0.787372,0.670524,0.510639,...,0.000000,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


...

/var/folders/l7/zj7563zj4nv2znvkzcllpvvr0000gn/T/ipykernel_56126/341764261.py:33: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  normalized_num_refs_per_pid = normalized_num_refs_per_pid.groupby(level=1, axis=1).mean()


,imported from Wikimedia project (P143),stated in (P248),retrieved (P813),reference URL (P854),Wikimedia import URL (P4656),based on heuristic (P887),inferred from (P3452)
"(income_class, nan)",,,,,,,
high_income,0.585438,0.240860,0.236967,0.123169,0.099689,0.092081,0.007734
low_income,0.571616,0.149127,0.198621,0.189065,0.144467,0.104477,0.007739
lower-middle_income,0.610352,0.135500,0.176706,0.156438,0.159050,0.105546,0.008967
upper-middle_income,0.612899,0.173980,0.172979,0.139106,0.144731,0.106749,0.005556


### For each decade and the top-50 relations that are equipped with the most references - normalized by claims with references

/var/folders/l7/zj7563zj4nv2znvkzcllpvvr0000gn/T/ipykernel_56126/341764261.py:33: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  normalized_num_refs_per_pid = normalized_num_refs_per_pid.groupby(level=1, axis=1).mean()


,reference URL (P854),retrieved (P813),stated in (P248),imported from Wikimedia project (P143),Wikimedia import URL (P4656),based on heuristic (P887),inferred from (P3452)
"(income_class, nan)",,,,,,,
high_income,0.557503,0.531397,0.478336,0.346698,0.217143,0.168944,0.012466
low_income,0.172194,0.255763,0.339320,0.151834,0.028027,0.282639,0.079084
lower-middle_income,0.230777,0.246901,0.361863,0.290152,0.086002,0.170892,0.038155
upper-middle_income,0.326315,0.360448,0.404399,0.240818,0.166242,0.168568,0.017746


In [17]:
calculate_all_stats(df_pid_counter, df_claim_counter, df_claim_with_ref_counter, df_num_refs, df_num_refs_per_pid, group_by=['income_class', 'decade'])

# Group by ['income_class', 'decade']

## Calculate percentage of persons with specific relation having references (there might be multiple claims per person)

person_counter                gender  \
income_class        decade                                         
high_income         1920s            89113  malefemalenon-binary   
                    1930s           186759  malefemalenon-binary   
                    1940s           251904  malefemalenon-binary   
                    1950s           292443  malefemalenon-binary   
                    1960s           279123  malefemalenon-binary   
                    1970s           261290  malefemalenon-binary   
                    1980s           240406  malefemalenon-binary   
                    1990s           179023  malefemalenon-binary   
                    2000s            66537  femalemalenon-binary   
                    2010s             1181            femalemale   
                    2020s              104            femalemale   
low_income          1920s              441            femalemale   
                    1930s             1380  malefemalenon-binary   
                    1940s             2078            malefemale   
                    1950s             3207  malefemalenon-binary   
                    1960s             2705  malefemalenon-binary   
                    1970s             2738  malefemalenon-binary   
                    1980s             3794  malefemalenon-binary   
                    1990s             3742  malefemalenon-binary   
                    2000s             1432            femalemale   
                    2010s               16            femalemale   
lower-middle_income 1920s             4554  malefemalenon-binary   
                    1930s            11169            malefemale   
                    1940s            13918  malefemalenon-binary   
                    1950s            18064  malefemalenon-binary   
                    1960s            16933  malefemalenon-binary   
                    1970s            16909  malefemalenon-binary   
                    1980s            21478  malefemalenon-binary   
                    1990s            18932  malefemalenon-binary   
                    2000s             5974  malefemalenon-binary   
                    2010s               60            femalemale   
                    2020s               12            malefemale   
upper-middle_income 1920s            11803  malefemalenon-binary   
                    1930s            27127  malefemalenon-binary   
                    1940s            34738  malefemalenon-binary   
                    1950s            52253  malefemalenon-binary   
                    1960s            66050  malefemalenon-binary   
                    1970s            67139  femalemalenon-binary   
                    1980s            71900  malefemalenon-binary   
                    1990s            59728  malefemalenon-binary   
                    2000s            17632  femalemalenon-binary   
                    2010s              257            malefemale   
                    2020s               38            malefemale   

                            VIAF cluster ID (P214)  \
income_class        decade                           
high_income         1920s                 0.359274   
                    1930s                 0.342725   
                    1940s                 0.316930   
                    1950s                 0.261921   
                    1960s                 0.235280   
                    1970s                 0.178288   
                    1980s                 0.090081   
                    1990s                 0.033867   
                    2000s                 0.068473   
                    2010s                 0.017782   
                    2020s                 0.038462   
low_income          1920s                 0.208617   
                    1930s                 0.216667   
                    1940s                 0.179981   
                    1950s                 0.126286   
                    1960s                 0.10831

## Calculate percentage of claims with specific relation having references

person_counter                gender  \
income_class        decade                                         
high_income         1920s            89113  malefemalenon-binary   
                    1930s           186759  malefemalenon-binary   
                    1940s           251904  malefemalenon-binary   
                    1950s           292443  malefemalenon-binary   
                    1960s           279123  malefemalenon-binary   
                    1970s           261290  malefemalenon-binary   
                    1980s           240406  malefemalenon-binary   
                    1990s           179023  malefemalenon-binary   
                    2000s            66537  femalemalenon-binary   
                    2010s             1181            femalemale   
                    2020s              104            femalemale   
low_income          1920s              441            femalemale   
                    1930s             1380  malefemalenon-binary   
                    1940s             2078            malefemale   
                    1950s             3207  malefemalenon-binary   
                    1960s             2705  malefemalenon-binary   
                    1970s             2738  malefemalenon-binary   
                    1980s             3794  malefemalenon-binary   
                    1990s             3742  malefemalenon-binary   
                    2000s             1432            femalemale   
                    2010s               16            femalemale   
lower-middle_income 1920s             4554  malefemalenon-binary   
                    1930s            11169            malefemale   
                    1940s            13918  malefemalenon-binary   
                    1950s            18064  malefemalenon-binary   
                    1960s            16933  malefemalenon-binary   
                    1970s            16909  malefemalenon-binary   
                    1980s            21478  malefemalenon-binary   
                    1990s            18932  malefemalenon-binary   
                    2000s             5974  malefemalenon-binary   
                    2010s               60            femalemale   
                    2020s               12            malefemale   
upper-middle_income 1920s            11803  malefemalenon-binary   
                    1930s            27127  malefemalenon-binary   
                    1940s            34738  malefemalenon-binary   
                    1950s            52253  malefemalenon-binary   
                    1960s            66050  malefemalenon-binary   
                    1970s            67139  femalemalenon-binary   
                    1980s            71900  malefemalenon-binary   
                    1990s            59728  malefemalenon-binary   
                    2000s            17632  femalemalenon-binary   
                    2010s              257            malefemale   
                    2020s               38            malefemale   

                            VIAF cluster ID (P214)  \
income_class        decade                           
high_income         1920s                 0.619888   
                    1930s                 0.595541   
                    1940s                 0.582791   
                    1950s                 0.554247   
                    1960s                 0.552836   
                    1970s                 0.506194   
                    1980s                 0.448773   
                    1990s                 0.463178   
                    2000s                 0.595556   
                    2010s                 0.420000   
                    2020s                 0.571429   
low_income          1920s                 0.505495   
                    1930s                 0.572797   
                    1940s                 0.545985   
                    1950s                 0.518566   
                    1960s                 0.56673

...

### For each decade and the top-50 relations that are equipped with the most references - normalized by claims with references

/var/folders/l7/zj7563zj4nv2znvkzcllpvvr0000gn/T/ipykernel_56126/341764261.py:33: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  normalized_num_refs_per_pid = normalized_num_refs_per_pid.groupby(level=1, axis=1).mean()


stated in (P248)  retrieved (P813)  \
(income_class, nan) (decade, nan)                                       
high_income         1920s                  0.459036          0.294534   
                    1930s                  0.455398          0.299898   
                    1940s                  0.460601          0.356550   
                    1950s                  0.419783          0.357051   
                    1960s                  0.396055          0.296580   
                    1970s                  0.389109          0.302100   
                    1980s                  0.406988          0.426125   
                    1990s                  0.443098          0.408264   
                    2000s                  0.365924          0.343547   
                    2010s                  0.205714          0.103504   
                    2020s                  0.050794          0.105590   
low_income          1920s                  0.072175          0.018142   
                    1930s                  0.200522          0.123265   
                    1940s                  0.164236          0.092859   
                    1950s                  0.275295          0.226460   
                    1960s                  0.264367          0.168673   
                    1970s                  0.248057          0.166231   
                    1980s                  0.326156          0.180779   
                    1990s                  0.235920          0.124540   
                    2000s                  0.086729          0.050747   
                    2010s                  0.006667          0.008696   
lower-middle_income 1920s                  0.250625          0.143567   
                    1930s                  0.235879          0.149060   
                    1940s                  0.304360          0.184440   
                    1950s                  0.252876          0.170947   
                    1960s                  0.297381          0.123195   
                    1970s                  0.419996          0.284863   
                    1980s                  0.280346          0.154792   
                    1990s                  0.261501          0.128723   
                    2000s                  0.195709          0.114750   
                    2010s                  0.026667          0.050725   
                    2020s                  0.011111          0.007246   
upper-middle_income 1920s                  0.193560          0.239425   
                    1930s                  0.307729          0.318908   
                    1940s                  0.341307          0.325580   
                    1950s                  0.354476          0.241918   
                    1960s                  0.420748          0.318045   
                    1970s                  0.413500          0.302674   
                    1980s                  0.297421          0.292433   
                    1990s                  0.272215          0.224594   
                    2000s                  0.340096          0.162348   
                    2010s                  0.165523          0.056599   
                    2020s                  0.072464          0.005671   

                                   imported from Wikimedia project (P143)  \
(income_class, nan) (decade, nan)                                           
high_income         1920s                                        0.209370   
                    1930s                                        0.252640   
                    1940s                                        0.186606   
                    1950s                                        0.198995   
                    1960s                                        0.207549   
                    1970s                                        0.223023   
                    1980s                                        0.314922   
                    1990s                               

### Average over all decades for the top-50 relations that are equipped with the most references - normalized by claims with references

,stated in (P248),retrieved (P813),imported from Wikimedia project (P143),reference URL (P854),Wikimedia import URL (P4656),based on heuristic (P887),inferred from (P3452)
"(income_class, nan)",,,,,,,
high_income,0.368409,0.299431,0.200088,0.236077,0.102110,0.110215,0.009493
low_income,0.188012,0.116039,0.130584,0.113554,0.024611,0.045806,0.052949
lower-middle_income,0.230586,0.137483,0.195559,0.136423,0.029692,0.101724,0.036745
upper-middle_income,0.289004,0.226200,0.178556,0.173584,0.078548,0.110193,0.032663


In [18]:
calculate_all_stats(df_pid_counter, df_claim_counter, df_claim_with_ref_counter, df_num_refs, df_num_refs_per_pid, group_by=['income_class', 'gender'])

# Group by ['income_class', 'gender']

## Calculate percentage of persons with specific relation having references (there might be multiple claims per person)

person_counter  \
income_class        gender                       
high_income         female              502510   
                    male               1342652   
                    non-binary            2721   
low_income          female                5175   
                    male                 16337   
                    non-binary              21   
lower-middle_income female               30195   
                    male                 97653   
                    non-binary             155   
upper-middle_income female              106807   
                    male                301337   
                    non-binary             521   

                                                                           decade  \
income_class        gender                                                          
high_income         female      1930s1980s1970s1940s1950s1960s1920s2000s1990s2...   
                    male        1930s1920s1980s1990s1970s1950s1960s1940s2000s2...   
                    non-binary      1950s1970s1990s1980s1960s2000s1940s1930s1920s   
low_income          female      2000s1970s1980s1950s1960s1940s1990s1920s1930s2...   
                    male        1980s1960s1970s1940s1950s1990s1930s2000s1920s2...   
                    non-binary                     1930s1950s1980s1990s1970s1960s   
lower-middle_income female      1980s1940s2000s1960s1990s1970s1950s1920s1930s2...   
                    male        1940s1960s1930s2000s1970s1980s1990s1950s1920s2...   
                    non-binary           1990s1980s1960s1970s1950s2000s1940s1920s   
upper-middle_income female      2000s1970s1940s1960s1980s1990s1950s1930s1920s2...   
                    male        1940s1960s1950s1970s1980s1990s1930s1920s2000s2...   
                    non-binary      1970s1960s1980s1930s1920s1950s1990s1940s2000s   

                                VIAF cluster ID (P214)  \
income_class        gender                               
high_income         female                    0.191606   
                    male                      0.223554   
                    non-binary                0.210952   
low_income          female                    0.074396   
                    male                      0.088694   
                    non-binary                0.142857   
lower-middle_income female                    0.085610   
                    male                      0.095041   
                    non-binary                0.058065   
upper-middle_income female                    0.059369   
                    male                      0.068873   
                    non-binary                0.080614   

                                member of political party (P102)  \
income_class        gender                                         
high_income         female                              0.036099   
                    male                                0.047012   
                    non-binary                          0.035649   
low_income          female                              0.067053   
                    male                                0.098182   
                    non-binary                          0.047619   
lower-middle_income female                              0.061434   
                    male                                0.112070   
                    non-binary                          0.038710   
upper-middle_income female                              0.239760   
                    male                                0.217813   
                    non-binary                          0.074856   

                                educated at (P69)  instance of (P31)  \
income_class        gender                                             
high_income         female               0.231792           0.395636   
                    male                 0.283370           0.496880   
                    non-binary           0.360162           0.339214   
low_income    

## Calculate percentage of claims with specific relation having references

person_counter  \
income_class        gender                       
high_income         female              502510   
                    male               1342652   
                    non-binary            2721   
low_income          female                5175   
                    male                 16337   
                    non-binary              21   
lower-middle_income female               30195   
                    male                 97653   
                    non-binary             155   
upper-middle_income female              106807   
                    male                301337   
                    non-binary             521   

                                                                           decade  \
income_class        gender                                                          
high_income         female      1930s1980s1970s1940s1950s1960s1920s2000s1990s2...   
                    male        1930s1920s1980s1990s1970s1950s1960s1940s2000s2...   
                    non-binary      1950s1970s1990s1980s1960s2000s1940s1930s1920s   
low_income          female      2000s1970s1980s1950s1960s1940s1990s1920s1930s2...   
                    male        1980s1960s1970s1940s1950s1990s1930s2000s1920s2...   
                    non-binary                     1930s1950s1980s1990s1970s1960s   
lower-middle_income female      1980s1940s2000s1960s1990s1970s1950s1920s1930s2...   
                    male        1940s1960s1930s2000s1970s1980s1990s1950s1920s2...   
                    non-binary           1990s1980s1960s1970s1950s2000s1940s1920s   
upper-middle_income female      2000s1970s1940s1960s1980s1990s1950s1930s1920s2...   
                    male        1940s1960s1950s1970s1980s1990s1930s1920s2000s2...   
                    non-binary      1970s1960s1980s1930s1920s1950s1990s1940s2000s   

                                VIAF cluster ID (P214)  \
income_class        gender                               
high_income         female                    0.506097   
                    male                      0.574208   
                    non-binary                0.473988   
low_income          female                    0.472973   
                    male                      0.537264   
                    non-binary                0.428571   
lower-middle_income female                    0.483811   
                    male                      0.532717   
                    non-binary                0.375000   
upper-middle_income female                    0.456318   
                    male                      0.501704   
                    non-binary                0.415842   

                                member of political party (P102)  \
income_class        gender                                         
high_income         female                              0.519547   
                    male                                0.511752   
                    non-binary                          0.602484   
low_income          female                              0.601386   
                    male                                0.724481   
                    non-binary                          1.000000   
lower-middle_income female                              0.634405   
                    male                                0.641162   
                    non-binary                          0.600000   
upper-middle_income female                              0.927289   
                    male                                0.871901   
                    non-binary                          0.684211   

                                educated at (P69)  instance of (P31)  \
income_class        gender                                             
high_income         female               0.526564           0.395504   
                    male                 0.587104           0.496759   
                    non-binary           0.587530           0.339089   
low_income    

...

### For each decade and the top-50 relations that are equipped with the most references - normalized by claims with references

/var/folders/l7/zj7563zj4nv2znvkzcllpvvr0000gn/T/ipykernel_56126/341764261.py:33: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  normalized_num_refs_per_pid = normalized_num_refs_per_pid.groupby(level=1, axis=1).mean()


stated in (P248)  reference URL (P854)  \
(income_class, nan) (gender, nan)                                           
high_income         female                 0.395909              0.375837   
                    male                   0.479606              0.376557   
                    non-binary             0.360156              0.237791   
low_income          female                 0.214248              0.190481   
                    male                   0.352922              0.162434   
                    non-binary             0.095833              0.052885   
lower-middle_income female                 0.290259              0.161133   
                    male                   0.367973              0.227793   
                    non-binary             0.090598              0.138856   
upper-middle_income female                 0.334612              0.201160   
                    male                   0.403846              0.296033   
                    non-binary             0.211795              0.159725   

                                   retrieved (P813)  \
(income_class, nan) (gender, nan)                     
high_income         female                 0.375421   
                    male                   0.466179   
                    non-binary             0.397070   
low_income          female                 0.218160   
                    male                   0.206495   
                    non-binary             0.059783   
lower-middle_income female                 0.164126   
                    male                   0.247166   
                    non-binary             0.056689   
upper-middle_income female                 0.275210   
                    male                   0.357591   
                    non-binary             0.181351   

                                   imported from Wikimedia project (P143)  \
(income_class, nan) (gender, nan)                                           
high_income         female                                       0.316050   
                    male                                         0.327158   
                    non-binary                                   0.212628   
low_income          female                                       0.094040   
                    male                                         0.156850   
                    non-binary                                   0.019531   
lower-middle_income female                                       0.223024   
                    male                                         0.292446   
                    non-binary                                   0.116506   
upper-middle_income female                                       0.233754   
                    male                                         0.234704   
                    non-binary                                   0.139904   

                                   Wikimedia import URL (P4656)  \
(income_class, nan) (gender, nan)                                 
high_income         female                             0.193898   
                    male                               0.197674   
                    non-binary                         0.130461   
low_income          female                             0.023287   
                    male                               0.028457   
                    non-binary                         0.000000   
lower-middle_income female                             0.028688   
                    male                               0.084443   
                    non-binary                         0.015460   
upper-middle_income female                             0.158731   
                    male                               0.158300   
                    non-binary                         0.071099   

                                   based on heuristic (P887)  \
(income_class, nan) (gender, nan)                              
high_income         female             

### Average over all decades for the top-50 relations that are equipped with the most references - normalized by claims with references

,stated in (P248),reference URL (P854),retrieved (P813),imported from Wikimedia project (P143),Wikimedia import URL (P4656),based on heuristic (P887),inferred from (P3452)
"(income_class, nan)",,,,,,,
high_income,0.411890,0.330062,0.412890,0.285279,0.174011,0.161605,0.010267
low_income,0.221001,0.135267,0.161479,0.090140,0.017248,0.104880,0.061329
lower-middle_income,0.249610,0.175927,0.155994,0.210659,0.042864,0.128322,0.026182
upper-middle_income,0.316751,0.218973,0.271384,0.202787,0.129377,0.122672,0.012217
